# Schudeler Query from Bronce to Silver for Items

Library

In [40]:
import pandas as pd
from pandas_gbq import read_gbq, to_gbq
import json

Project Info

In [41]:
project_id = "big-data-lol"
dataset_id_from = "Bronze_LOL_Dataset"
dataset_id_to = "Silver_LOL_Dataset"
table_id_from = "bronze_batch_items"
table_id_to = "silver_batch_items"

Lol current patch

In [42]:
current_patch = "14.12.2"

Query

In [43]:
query = f'''
SELECT DATA
FROM {project_id}.{dataset_id_from}.{table_id_from}
WHERE TIMESTAMP_TRUNC(_PARTITIONTIME, DAY) = TIMESTAMP("2024-06-16")
AND Patch = "{current_patch}"
'''

Query in BigQuery Dataset

In [44]:
df = read_gbq(query, project_id=project_id, dialect='standard')


Downloading: 100%|██████████|


In [45]:
df.head(1)

,DATA
0,"{""name"": ""Botas"", ""description"": ""<mainText><s..."


In [46]:
def json_to_dataframe(json_str):
    # Cargar el JSON en un diccionario
    json_data = json.loads(json_str)

    # Crear un DataFrame a partir del diccionario
    df = pd.DataFrame.from_dict([json_data])

    return df



In [47]:
df_processed = pd.concat([json_to_dataframe(row) for row in df['DATA']], ignore_index=True)


In [48]:
df_processed.columns.tolist()

['name',
 'description',
 'colloq',
 'plaintext',
 'into',
 'image',
 'gold',
 'tags',
 'maps',
 'stats',
 'from',
 'depth',
 'inStore',
 'effect',
 'consumed',
 'requiredAlly',
 'stacks',
 'hideFromAll',
 'consumeOnFull',
 'specialRecipe',
 'requiredChampion']

In [49]:
df_processed.head(1)

,name,description,colloq,plaintext,into,image,gold,tags,maps,stats,...,depth,inStore,effect,consumed,requiredAlly,stacks,hideFromAll,consumeOnFull,specialRecipe,requiredChampion
0,Botas,<mainText><stats><attention>25</attention> de ...,;Boots of Speed,Aumenta ligeramente la Velocidad de Movimiento.,"[3005, 3047, 3117, 3006, 3009, 3010, 3020, 311...","{'full': '1001.png', 'sprite': 'item0.png', 'g...","{'base': 300, 'purchasable': True, 'total': 30...",[Boots],"{'11': True, '12': True, '21': True, '22': Fal...",{'FlatMovementSpeedMod': 25},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
from google.cloud import bigquery
schema = [
  bigquery.SchemaField('name', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('description', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('colloq', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('plaintext', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('into', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('image', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('gold', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('tags', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('maps', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('stats', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('from', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('depth', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('inStore', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('effect', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('consumed', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('requiredAlly', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('stacks', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('hideFromAll', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('consumeOnFull', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('specialRecipe', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('requiredChampion', 'STRING', 'NULLABLE')
]
client = bigquery.Client(project=project_id)

validated_path = '/tmp/validated_' + "champs_to_silver.csv"
# se transforma el df a csv
df_processed.to_csv(validated_path, index=False)

print(validated_path)
# Cargar los datos validados en BigQuery
dataset_ref = client.dataset(dataset_id_to)
table_ref = dataset_ref.table(table_id_to)

#configuracion del job
job_config = bigquery.LoadJobConfig()

job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.schema = schema
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE


with open(validated_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()


/tmp/validated_champs_to_silver.csv


LoadJob<project=big-data-lol, location=US, id=6dbd22b9-f9e3-4f5d-b30d-1007b1f9b631>